### Coursework 2

In this coursework you will be aiming to complete two classification tasks. 
Both the classification tasks relate to text classification tasks. 

One task is to be solved using Support Vector Machines. The other has to be solved using Boosting.

The specific tasks and the marking for the various tasks are provided in the notebook. Each task is expected to be accompanied by a lab-report. Each task can have a concise lab report that is maximum of one page in an A4 size. You will be expected to submit your Jupyter Notebook and all lab reports as a single zip file. You could have additional functions implemented that you require for carrying out each task.


#### Task 1

In this task, you need to obtain sentiment analysis for the provided dataset. The dataset consists of movie reviews with the sentiments being provided. The sentiments are either positive or negative. You need to train an SVM based classifier to obtain train and check on the sample test dataset provided. The method will be evaluated also against an external test set. Please do not hardcode any dimensions or number of samples while writing the code. It should be possible to automate the testing and hardcoding values does not allow for automated testing. 

You are allowed to use scikit-learn to implement the SVM. However, you are expected to write your own kernels.

You are allowed to use the existing library functions such as scikit-learn or numpy for obtaining the SVM. The main idea is to analyse the dataset using different kind of kernels. You are also supposed to write your own custom text kernels. Refer to the documentation provided [here](https://scikit-learn.org/stable/modules/svm.html) at 1.4.6.2 and an example [here](https://scikit-learn.org/stable/auto_examples/svm/plot_custom_kernel.html) for writing your own kernels.

Details regarding the marking have been provided in the coursework specification file. Ensure that the code can be run with different test files. 

#### Process the text and obtain a bag of words-based features

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer
from bs4 import BeautifulSoup
import re
import nltk

In [2]:
train_file = pd.read_csv('movie_review_train.csv')
test_file = pd.read_csv('movie_review_test.csv')

In [3]:
train_file.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
tokenizer=ToktokTokenizer()

def normalize_data(train_file, test_file):
    tokenizer= ToktokTokenizer()
    #Setting English stopwords
    stopword_list=nltk.corpus.stopwords.words('english')

    def strip_html(text):
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()

    #Removing the square brackets
    def remove_between_square_brackets(text):
        return re.sub('\[[^]]*\]', '', text)

    #Removing the noisy text
    def denoise_text(text):
        text = strip_html(text)
        text = remove_between_square_brackets(text)
        return text

    train_file['review']= train_file['review'].apply(denoise_text)
    test_file['review']= test_file['review'].apply(denoise_text)


    #Define function for removing special characters
    def remove_special_characters(text, remove_digits=True):
        pattern=r'[^a-zA-z0-9\s]'
        text=re.sub(pattern,'',text)
        return text
    #Apply function on review column
    train_file['review'] = train_file['review'].apply(remove_special_characters)
    test_file['review'] = test_file['review'].apply(remove_special_characters)

    # remove whitespace
    train_file['review'] = train_file['review'].apply(lambda x: x.strip())
    test_file['review'] = test_file['review'].apply(lambda x: x.strip())


    # convert to lower
    train_file['review'] = train_file['review'].apply(lambda x: x.lower())
    test_file['review'] = test_file['review'].apply(lambda x: x.lower())

    #Stemming the text
    def simple_stemmer(text):
        ps= nltk.porter.PorterStemmer()
        text= ' '.join([ps.stem(word) for word in text.split()])
        return text
    #Apply function on review column
    train_file['review'] = train_file['review'].apply(simple_stemmer)
    test_file['review'] = test_file['review'].apply(simple_stemmer)

    # stop=set(stopwords.words('english'))
    # print(stop)

#removing the stopwords
    def remove_stopwords(text, is_lower_case=False):
        tokens = tokenizer.tokenize(text)
        tokens = [token.strip() for token in tokens]
        if is_lower_case:
            filtered_tokens = [token for token in tokens if token not in stopword_list]
        else:
            filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
        filtered_text = ' '.join(filtered_tokens)
        return filtered_text

#Apply function on review column
    train_file['review']=train_file['review'].apply(remove_stopwords)
    test_file['review']=test_file['review'].apply(remove_stopwords)


    X_train = train_file['review']
    X_test = test_file['review']

    return X_train, X_test

In [5]:
def extract_bag_of_words_train_test(train_file, test_file):
    import numpy as np
    import nltk
    # Read the CSV file and extract Bag of Words Features
    train_file = pd.read_csv(train_file)
    test_file = pd.read_csv(test_file)

    X_train, X_test = normalize_data(train_file,test_file)

    #Count vectorizer for bag of words
    cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
    #transformed train reviews
    X_train = cv.fit_transform(X_train)
    #transformed test reviews
    X_test = cv.transform(X_test)

    #labeling the sentient data
    lb = LabelBinarizer(neg_label= -1)
    #transformed sentiment data
    y_train = lb.fit_transform(train_file['sentiment'])
    y_test = lb.fit_transform(test_file['sentiment'])

    return (X_train,y_train,X_test,y_test)

In [9]:
# Remove this later
class SVMClassifier:
    """Placeholder SVM classifier SGD for testing and analysis purposes.
    """
    def __init__(self):
        from sklearn.linear_model import SGDClassifier
        self.svm=SGDClassifier(loss='hinge',max_iter=5000,random_state=42)

    def fit(self, X, y):
        y = y.ravel()
        self.svm = self.svm.fit(X,y)
        return self.svm

    def predict(self, X):
        return self.svm.predict(X)


# Keep this one
# TODO: Customize
class SVMClassifierCustom:
    def __init__(self):
        import numpy as np
        from sklearn import svm
        # We create a Linear Kernel with the following hyperparameters
        self.C = 10
    # define your own kernel here
    # Refer to the documentation here: https://scikit-learn.org/stable/auto_examples/svm/plot_custom_kernel.html
    def fit(self, X,y):
        # training of the SVM
        # Ensure you call your own defined kernel here
        return
    def predict(self, X):
        # prediction routine for the SVM
        return    

### Test function that will be called to evaluate your code. Separate test dataset will be provided

Do not modify the code below. Please write your code above such that it can be evaluated by the function below. You can modify your code above such that you obtain the best performance through this function.

In [10]:
def test_func_svm(dataset_train, dataset_test):
    from sklearn.metrics import accuracy_score
    (X_train, Y_train, X_test, Y_test) = extract_bag_of_words_train_test(dataset_train, dataset_test)
    sc = SVMClassifier()
    sc.fit(X_train, Y_train)
    Y_Pred = sc.predict(X_test)
    acc = accuracy_score(Y_test, Y_Pred)
    print("Accuracy:",acc)
    return acc

In [11]:
acc = test_func_svm("movie_review_train.csv", "movie_review_test.csv")

/Users/kuba/miniforge3/envs/pytorch/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


{'mightn', "it's", 'will', "won't", 'about', 'have', 'ain', 'couldn', 'for', 'itself', 'which', 'at', "mustn't", 've', 'by', 'was', 'there', 'didn', 'being', "doesn't", 'myself', "shouldn't", 'been', 'from', 'under', 'to', 'only', 'they', "you'll", 'not', 'such', "that'll", 'before', "didn't", 'hadn', 'do', 'into', 'now', 'here', 'll', 'what', 'did', 'during', 'doing', 'themselves', 'hasn', "hasn't", 'shan', 'below', 'don', 'of', 'am', 'but', 'our', 'then', 'd', 'few', 'until', 'or', 'against', 'why', 'no', "wasn't", 'wouldn', 'your', 'over', "should've", 'can', 'above', 'out', 'isn', 'that', 'too', 'each', 'y', "haven't", 'does', 'his', 'most', "needn't", 'down', 'their', 'the', 'very', 's', 'just', 'aren', 'won', 'them', 'so', 'my', 'you', 'himself', "wouldn't", "couldn't", 'are', 'm', 'had', 'on', 'me', 'when', 'all', 'yours', 'these', 'should', "hadn't", 'and', 'weren', 'a', 'theirs', 'while', "isn't", 'he', 'those', "aren't", 'o', 'again', 'some', 're', 'off', 'we', 'were', 'as', 

### Task 2

In this task you need to implement a boosting based classifier that can be used to classify the images. 

Details regarding the marking for the coursework are provided in the coursework specification file. Please ensure that your code will work with a different test file than the one provided with the coursework.

Note that the boosting classifier you implement can include decision trees from scikit-learn or your own decision trees. Use the same sentiment analysis dataset for evaluation.

In [4]:
class BoostingClassifier:
    # You need to implement this classifier. 
    def __init__(self):
        import numpy as np
        #implement initialisation
        self.some_paramter=1
    def fit(self, X,y):
        from sklearn.tree import DecisionTreeClassifier
        import numpy as np
        #implement training of the boosting classifier
        return 
    def predict(self, X):
        # implement prediction of the boosting classifier
        return

### Test function that will be called to evaluate your code. Separate test dataset will be provided

Do not modify the code below. Please write your code above such that it can be evaluated by the function below. You can modify your code above such that you obtain the best performance through this function. 

In [5]:
def test_func_boosting(dataset_train, dataset_test):
    from sklearn.metrics import accuracy_score    
    (X_train, Y_train, X_test, Y_test) = extract_bag_of_words_train_test(dataset_train, dataset_test)
    bc = BoostingClassifier()
    bc.fit(X_train, Y_train)
    Y_Pred = bc.predict(X_test)    
    acc = accuracy_score(Y_test, Y_Pred)
    return acc

In [ ]:
acc = test_func_boosting("movie_review_train.csv", "movie_review_test.csv")